In [ ]:
import brightway2 as bw
from pathlib import Path
from gsa_framework import LCAModel
from gsa_framework.validation import Validation
from gsa_framework.utils_setac_lca import *
from gsa_framework.utils import read_hdf5_array

In [ ]:
path_base = Path('/data/user/kim_a/setac_gsa/')
path_merlin = path_base / "merlin"
filepath_scores_lsa_dict = path_merlin / "scores_lsa_dict.pickle"
filepath_params_yes_0 = path_merlin / "params_yes_0.pickle"
filepath_params_yes_6 = path_merlin / "params_yes_6.pickle"

with open(filepath_scores_lsa_dict, 'rb') as f:
    scores_lsa_dict = pickle.load(f)
    
with open(filepath_params_yes_0, 'rb') as f:
    params_yes_0 = pickle.load(f)
    
with open(filepath_params_yes_6, 'rb') as f:
    params_yes_6 = pickle.load(f)

# Influential Y after refactoring of validation and regression

In [ ]:
# path_base = Path(
#     "/Users/akim/PycharmProjects/gsa_framework/dev/write_files/paper_gsa/"
# )
path_base = Path('/data/user/kim_a/')

# LCA model
bw.projects.set_current("GSA for setac")
co = bw.Database("CH consumption 1.0")
demand_act = [act for act in co if "Food" in act["name"]][0]
demand = {demand_act: 1}
method = ("IPCC 2013", "climate change", "GTP 100a")
write_dir = path_base / "setac_gsa"
# var_threshold = 100
lca_model = LCAModel(demand, method, write_dir)

# Define some variables
# seed 10447 refers to default_x_rescaled=amounts
# seed 923458 referse to default_x_rescaled=model.default_x_rescaled
seed = 10447 
num_params = len(lca_model)
iterations_validation = 2000
bin_min, bin_max = 2300, 3300

tech_params = lca_model.lca.tech_params
amounts = tech_params[tech_params["uncertainty_type"]>1]["amount"]

validation = Validation(
    lca_model,
    iterations=iterations_validation,
    seed=seed,
    default_x_rescaled=lca_model.default_x_rescaled, #TODO change??
    write_dir=write_dir,
)

# 1. Validation plot base_Y
validation.plot_histogram_base_Y(bin_min=bin_min, bin_max=bin_max, save_fig=False)

In [ ]:
# 2. Influential_Y after LSA_3
params_yes_inf = params_yes_6

params_yes_inf_where = np.array([np.where(params_yes_0==p)[0][0] for p in params_yes_inf])
parameter_choice = params_yes_inf_where
influential_Y = validation.get_influential_Y_from_parameter_choice(parameter_choice)
validation.plot_histogram_base_Y_influential_Y(
    influential_Y, tag=len(parameter_choice), save_fig=False, bin_min=bin_min, bin_max=bin_max
)

# 3. After regression, 8722 parameters
# model_seed = 3333
# path_model_dir = write_dir / "regression" / "{}_model".format(model_seed)
# model, params_yes_xgboost, _ = get_xgboost_params(path_model_dir, params_yes_0)
# params_yes_inf = params_yes_xgboost
# params_yes_inf_where = np.array([np.where(params_yes_0==p)[0][0] for p in params_yes_inf])

# parameter_choice = params_yes_inf_where
# influential_Y = validation.get_influential_Y_from_parameter_choice(parameter_choice)

# validation.plot_histogram_base_Y_influential_Y(
#     influential_Y, tag=len(parameter_choice), save_fig=False, bin_min=bin_min, bin_max=bin_max
# )
# filepath_influential_Y = (
#     write_dir / "arrays" / validation.create_influential_model_output_filepath(tag)
# )

In [ ]:
# 4. After regression, 60 parameters
model_seed = 3333
path_model_dir = write_dir / "regression" / "{}_model".format(model_seed)
model, params_yes_xgboost, _ = get_xgboost_params(path_model_dir, params_yes_0)

num_influential = 60
params_yes_inf = params_yes_xgboost[:num_influential]
params_yes_inf_where = np.array([np.where(params_yes_0==p)[0][0] for p in params_yes_inf])

parameter_choice = params_yes_inf_where
influential_Y = validation.get_influential_Y_from_parameter_choice(parameter_choice)

validation.plot_histogram_base_Y_influential_Y(
    influential_Y, tag=len(parameter_choice), save_fig=False, bin_min=bin_min, bin_max=bin_max
)